## Імпорт бібліотек

In [1]:
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.image import resize
from keras.models import load_model
import numpy as np
import skimage
import cv2
import os
import re
from PIL import Image

## Тестові фото

In [3]:
dir_path = r'C:\Users\kuzik\Desktop\photos'
list_files = os.listdir(dir_path)
list_files
files  = []
for filename in list_files:
    if filename != 'desktop.ini':
        files.append(re.sub(r'\\', r'/', dir_path + "/" + filename))

## Функція обробки фото і передбачення

In [4]:
def img_preproces(img, expand_dims = False, copy = False, preprocess_img = False):
    if expand_dims == True:
        img = np.expand_dims(img, axis=0)
    if copy == True:
        img = np.copy(img)
    if preprocess_img == True:
        img = preprocess_input(img).astype("float32") / 255
    return img

def tf_resize(img):
    img = resize(img, (32, 32))
    img = img_to_array(img)
    img = img_preproces(img, expand_dims = True, copy = True, preprocess_img = True)
    return img

def cv2_resize(img):
    img = cv2.resize(img, (32, 32), interpolation = cv2.INTER_AREA)
    img = img_preproces(img, expand_dims = True, copy = True, preprocess_img = True)
    return img

def si_resize(img):
    img = skimage.transform.resize(img, (32, 32, 3), anti_aliasing=True)
    img = img_preproces(img, expand_dims = True)
    return img

def pillow_resize(img):
    img = Image.open(img)
    img = img.resize((32, 32))
    img = img_preproces(img, expand_dims = True, copy = True, preprocess_img = True)
    return img
    

def si_rescale(img):
    for_rescale = list(img.shape)
    for_rescale.sort()
    scale_rating = 32 / for_rescale[1]
    img = skimage.transform.rescale(skimage.transform.resize(img, (for_rescale[1], for_rescale[1]), 3), scale_rating, multichannel=True)
    img = img_preproces(img, expand_dims = True)
    return img

def image_classify(filename, processor, model):
    print(filename)
    if processor == 'tf':
        img = load_img(filename)
        img = tf_resize(img)
    elif processor == 'cv2':
        img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        img = cv2_resize(img)
    elif processor == 'si_resize':
        img = skimage.io.imread(filename)
        img = si_resize(img)
    elif processor == 'si_rescale':
        img = skimage.io.imread(filename)
        img = si_rescale(img)
    elif processor == 'pillow':
        img = pillow_resize(filename)
    
    features = model.predict(img)
    top = []
    top_num = 3
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    for request in range(1):  # num of foto or objects on foto
        classes_request = classes
        features_request = features[0]
        rang = 1
        for i in range(top_num):
            pred = np.argmax(features_request)
            top.append([rang, classes_request[pred], round(features_request[np.argmax(features_request)], 4)])
            features_request = np.delete(features_request, np.argmax(features_request))
            classes_request.remove(classes_request[pred])
            rang +=1
    return top

## Перевірка роботи моделі з стисканням від tensorflow та cv2

In [5]:
def obj_on_file(file):
    file_name = file.split("/")[-1]
    file_name = file_name.split(".")[0:-1]
    find = re.match(r'\D+', str(file_name)).group()[2:]
    return find

def testing(path, processor, model, print_result = False):
    result = []
    obj = []
    for file in path:
        o = obj_on_file(file)
        predict_img = image_classify(file, processor, model = model)
        obj.append(o)
        result.append(predict_img[0][1])
        
        if print_result == True:
            print("")
            print("Must be: ", obj)
            print(predict_img)
        
    return result, obj

def accuracy_of_model(path, processor, model, print_result = False):
    result_test, answer_test = testing(path = path, processor = processor, model = model, print_result = print_result)
    list_of_results = []
    for num in range(len(result_test)):
        if result_test[num] == answer_test[num]:
            list_of_results.append(1)
        else:
            list_of_results.append(0)
    return list_of_results
    

def compare_processors(path, model, processor=[], print_result=False):
    result = []
    for el in processor:
        model_accuracy = accuracy_of_model(path, model = model, processor = el, print_result = print_result)
        accuracy_percent = model_accuracy.count(1) / len(model_accuracy) * 100
        result.append(f'Using {el} for resize photo, prediction accuracy of your photo set is: {accuracy_percent} %')
    return result

## Завантаження моделі

In [6]:
# model = load_model("C:/Users/kuzik/Desktop/project3/models/model_from_Susana_v2.h5")
# model.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_from_Susana_v2.h5")

# model1_1 = load_model("C:/Users/kuzik/Desktop/project3/models/model_from_Susana.h5")
# model1_1.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_from_Susana.h5")

# model2 = load_model("C:/Users/kuzik/Desktop/project3/models/model_from_mentor.h5")
# model2.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_from_mentor.h5")

# model3 = load_model("C:/Users/kuzik/Desktop/project3/models/base_model.h5")
# model3.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_base_model.h5")

# model4 = load_model("C:/Users/kuzik/Desktop/project3/models/model_from_Susana_v3.h5")
# model4.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_from_Susana_v3.h5")

# model5 = load_model("C:/Users/kuzik/Desktop/project3/models/model_from_mentor_v2.h5")
# model5.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_from_mentor_v2.h5")

# model6 = load_model("C:/Users/kuzik/Desktop/project3/models/model_from_mentor_v3.h5")
# model6.load_weights("C:/Users/kuzik/Desktop/project3/models/weights_from_mentor_v3.h5")

In [7]:
# s = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'pillow'], model=model) # 'tf', 'cv2','si_resize','si_rescale'
# s_1 = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'pillow'], model=model1_1) # 'si_rescale'
# m = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'pillow'], model=model2) # 'si_rescale'
# b = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'pillow'], model=model3) # 'si_rescale'
# s_3 = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'pillow'], model=model4) # 'si_rescale'
# m_2 = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'pillow'], model=model5) # 'si_rescale'
# m_3 = compare_processors(path=files, processor=['tf', 'cv2', 'si_resize', 'pillow'], model=model6) # 'si_rescale'

C:/Users/kuzik/Desktop/photos/airplane1.jpeg
1/1 [==============================] - 29s 29s/step
C:/Users/kuzik/Desktop/photos/airplane10.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/airplane11.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/airplane12.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/airplane13.jpg
1/1 [==============================] - 0s 37ms/step
C:/Users/kuzik/Desktop/photos/airplane14.jpg
1/1 [==============================] - 0s 29ms/step
C:/Users/kuzik/Desktop/photos/airplane15.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/airplane16.jpg
1/1 [==============================] - 0s 31ms/step
C:/Users/kuzik/Desktop/photos/airplane17.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/airplane18.jpg
1/1 [==============================] - 0s 37ms/step
C:/Users/kuzik/Desktop/photos/

1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/bird19.jpg
1/1 [==============================] - 0s 31ms/step
C:/Users/kuzik/Desktop/photos/bird2.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/bird20.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/bird21.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/bird22.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/bird23.jpg
1/1 [==============================] - 0s 43ms/step
C:/Users/kuzik/Desktop/photos/bird24.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/bird25.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/bird26.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/bird27.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desk

1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/cat8.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/cat9.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer1.jfif
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/deer10.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer11.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer12.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer13.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer14.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer15.jpg
1/1 [==============================] - 0s 33ms/step
C:/Users/kuzik/Desktop/photos/deer16.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop

1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/dog44.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/dog45.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/dog46.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/dog47.jpg
1/1 [==============================] - 0s 32ms/step
C:/Users/kuzik/Desktop/photos/dog48.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/dog49.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/dog5.jpg
1/1 [==============================] - 0s 38ms/step
C:/Users/kuzik/Desktop/photos/dog50.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/dog6.jpg
1/1 [==============================] - 0s 16ms/step
C:/Users/kuzik/Desktop/photos/dog7.jpg
1/1 [==============================] - 0s 38ms/step
C:/Users/kuzik/Desktop/photos/d

1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/horse26.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/horse27.jpg
1/1 [==============================] - 0s 30ms/step
C:/Users/kuzik/Desktop/photos/horse28.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/horse29.jpg
1/1 [==============================] - 0s 32ms/step
C:/Users/kuzik/Desktop/photos/horse3.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/horse30.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/horse31.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/horse32.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/horse33.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/horse34.jpg
1/1 [==============================] - 0s 36ms/step
C:/Users/

...

1/1 [==============================] - 0s 36ms/step
C:/Users/kuzik/Desktop/photos/bird8.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/bird9.jpg
1/1 [==============================] - 0s 32ms/step
C:/Users/kuzik/Desktop/photos/cat1.jfif
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/cat10.jpg
1/1 [==============================] - 0s 38ms/step
C:/Users/kuzik/Desktop/photos/cat11.jpg
1/1 [==============================] - 0s 19ms/step
C:/Users/kuzik/Desktop/photos/cat12.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/cat13.jpg
1/1 [==============================] - 0s 42ms/step
C:/Users/kuzik/Desktop/photos/cat14.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/cat15.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/cat16.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photo

1/1 [==============================] - 0s 34ms/step
C:/Users/kuzik/Desktop/photos/deer42.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/deer43.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/deer44.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer45.jpg
1/1 [==============================] - 0s 19ms/step
C:/Users/kuzik/Desktop/photos/deer46.jpg
1/1 [==============================] - 0s 33ms/step
C:/Users/kuzik/Desktop/photos/deer47.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/deer48.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/deer49.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/deer5.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/deer6.jpg
1/1 [==============================] - 0s 19ms/step
C:/Users/kuzik/Deskt

1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/frog34.jpg
1/1 [==============================] - 0s 30ms/step
C:/Users/kuzik/Desktop/photos/frog35.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/frog36.jpg
1/1 [==============================] - 0s 31ms/step
C:/Users/kuzik/Desktop/photos/frog37.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/frog38.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/frog39.jpg
1/1 [==============================] - 0s 21ms/step
C:/Users/kuzik/Desktop/photos/frog4.jpg
1/1 [==============================] - 0s 19ms/step
C:/Users/kuzik/Desktop/photos/frog40.jpg
1/1 [==============================] - 0s 42ms/step
C:/Users/kuzik/Desktop/photos/frog41.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/frog42.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desk

1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/ship14.jpg
1/1 [==============================] - 0s 40ms/step
C:/Users/kuzik/Desktop/photos/ship15.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/ship16.jpg
1/1 [==============================] - 0s 33ms/step
C:/Users/kuzik/Desktop/photos/ship17.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/ship18.jpg
1/1 [==============================] - 0s 17ms/step
C:/Users/kuzik/Desktop/photos/ship19.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/ship2.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/ship20.jpg
1/1 [==============================] - 0s 53ms/step
C:/Users/kuzik/Desktop/photos/ship21.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/ship22.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desk

1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/truck49.jpg
1/1 [==============================] - 0s 29ms/step
C:/Users/kuzik/Desktop/photos/truck5.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/truck50.jpg
1/1 [==============================] - 0s 19ms/step
C:/Users/kuzik/Desktop/photos/truck6.jpg
1/1 [==============================] - 0s 32ms/step
C:/Users/kuzik/Desktop/photos/truck7.jpg
1/1 [==============================] - 0s 18ms/step
C:/Users/kuzik/Desktop/photos/truck8.jpg
1/1 [==============================] - 0s 19ms/step
C:/Users/kuzik/Desktop/photos/truck9.jpg
1/1 [==============================] - 0s 19ms/step


# Test on files .png

## Susana's model

In [48]:
for el in s_1:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 25.0 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 61.36363636363637 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 57.27272727272727 %


In [43]:
for el in s:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 24.09090909090909 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 58.63636363636363 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 67.27272727272727 %


In [7]:
for el in s_3:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 43.18181818181818 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 67.56198347107438 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 76.65289256198346 %
Using pillow for resize photo, prediction accuracy of your photo set is: 60.74380165289256 %


## Mentor's Model

In [7]:
for el in m:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 33.47107438016529 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 48.553719008264466 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 45.04132231404959 %
Using pillow for resize photo, prediction accuracy of your photo set is: 42.768595041322314 %


In [8]:
for el in m_2:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 24.380165289256198 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 22.31404958677686 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 64.25619834710744 %
Using pillow for resize photo, prediction accuracy of your photo set is: 25.0 %


In [8]:
for el in m_3:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 15.082644628099173 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 16.94214876033058 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 60.9504132231405 %
Using pillow for resize photo, prediction accuracy of your photo set is: 18.388429752066116 %


## Base Model

In [45]:
for el in b:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 42.72727272727273 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 48.63636363636364 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 42.27272727272727 %


# Test on oll files

## Susana's model

In [17]:
for el in s_1:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 46.48760330578512 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 64.0495867768595 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 75.41322314049587 %
Using pillow for resize photo, prediction accuracy of your photo set is: 60.12396694214877 %


In [18]:
for el in s:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 47.31404958677686 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 68.18181818181817 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 78.92561983471074 %
Using pillow for resize photo, prediction accuracy of your photo set is: 65.08264462809917 %


## Mentor's Model

In [19]:
for el in m:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 33.47107438016529 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 48.553719008264466 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 45.04132231404959 %
Using pillow for resize photo, prediction accuracy of your photo set is: 42.768595041322314 %


## Base Model

In [20]:
for el in b:
    print(el)

Using tf for resize photo, prediction accuracy of your photo set is: 44.83471074380165 %
Using cv2 for resize photo, prediction accuracy of your photo set is: 57.231404958677686 %
Using si_resize for resize photo, prediction accuracy of your photo set is: 55.99173553719008 %
Using pillow for resize photo, prediction accuracy of your photo set is: 45.66115702479338 %
